In [1]:
import persistence_homology as ph

In [2]:
import csv

In [3]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output

In [4]:
folder_path = './lung_segmentations/segmentation-10'
edges_path = '/edges.csv'
vertices_path = '/vertices.csv'

In [5]:
vertices = read_csv(folder_path + vertices_path)
edges = read_csv(folder_path + edges_path)

In [6]:
direction = [0,0,1]

In [7]:
points = [[vertex, [0,0,0]] for vertex in vertices]
pre_edges = edges
pre_formatted_edges = []
for edge in pre_edges:
    x,y = edge
    pre_formatted_edges.append([[x, y], [1,1,1]])
pre_vertices = ph.append_height_vertices(direction, points)
vertices = ph.format_vertices(pre_vertices)
edges = ph.format_edges(vertices, pre_formatted_edges)
pre_graph = ph.process_graph(vertices, edges, direction)
graph = pre_graph['signed_graph']
original_to_new_indices = pre_graph['index_translation']
new_to_original_indices = {v: k for k, v in original_to_new_indices.items()}
filtration = ph.group_events_by_height(graph[0], graph[1])


# Filtration Algorithm Start

In [8]:
uf = ph.UnionFind(graph[0])

In [9]:
# Horizontal Step
def horizontal_step(edges: list, uf):
    for edge in edges:
        x ,y = edge['vertices']
        uf.union(x, y)
        
def vertical_step(edges: list, components: dict, mergers: dict, uf):
    for edge in edges:
        x ,y = edge['vertices']
        roots = uf.union(x, y)
        rootX = roots[x]['root']
        rootY = roots[y]['root']
        rankX = roots[x]['rank']
        rankY = roots[y]['rank']
        if rootX != rootY:
            if rootX in components and rootY in components:
                if rankY > rankX:
                    mergers[rootY]=max(edge['height'])
                    del components[rootY]
                elif rankX > rankY:
                    mergers[rootX]=max(edge['height'])
                    del components[rootX]
                else:
                    mergers[rootY]=max(edge['height'])
                    del components[rootY]
                    
def compute_components(vertices, old_components, uf):
    components = old_components
    for vertex in vertices:
        node = vertex['new_index']
        root = uf.find(node)  # This finds the representative of the component containing 'node'
        if root not in components:
            components[root] = []
        if node not in components[root]:
            components[root].append(node)
    return components

def compute_new_births(vertices, uf):
    new_components = []
    for vertex in vertices:
        node = vertex['new_index']
        root = uf.find(node)  # This finds the representative of the component containing 'node'
        if root == node:
            new_components.append(vertex)
    return new_components

def compute_intervals(births, mergers):
    """birth {'coordinates': [284.0, 315.0, 93.5], 'original_index': 18104, 'new_index': 0, 'height': 93.5, 'sign': 0}
    merger 5 98.5"""
    intervals = []
    for birth in births:
        left_bound = birth['height']
        right_bound = -1
        index = birth['new_index']
        if index in mergers.keys():
            right_bound = mergers[index]
        intervals.append([left_bound, right_bound])
    return intervals
    
def length_of_interval(interval):
    if interval[1] == -1:
        return -1
    else:
        return interval[1] - interval[0]

In [10]:
def compute_persistence(filtration):
    total_components = {}
    mergers = {}
    total_vertices = []
    births = []

    for height, stage in filtration.items():
        horizontal_edges = stage['horizontal_edges']
        vertical_edges = stage['vertical_edges']
        vertices = stage['points']
    
        # We join the horizontal edges
        horizontal_step(horizontal_edges)
        
        # We join the vertical edges
        vertical_step(vertical_edges, total_components, mergers)
        total_vertices.extend(vertices)
    
        # We compute the new components
        current_components = compute_components(total_vertices, total_components)
        total_components = current_components
    
        # We compute the horizontal connected components at a given stage, the unmerged are new components
        new_births = compute_new_births(vertices)
        births.extend(new_births)
    
    return total_components, mergers, total_vertices, births

In [11]:
total_components, mergers, total_vertices, births = compute_persistence(filtration)

In [12]:
print(births)

[{'coordinates': [284.0, 315.0, 93.5], 'original_index': 18104, 'new_index': 0, 'height': 93.5, 'sign': 0}, {'coordinates': [279.0, 299.0, 94.5], 'original_index': 17293, 'new_index': 5, 'height': 94.5, 'sign': 0}, {'coordinates': [306.0, 319.0, 95.5], 'original_index': 24411, 'new_index': 47, 'height': 95.5, 'sign': 0}, {'coordinates': [299.0, 304.0, 96.5], 'original_index': 22256, 'new_index': 129, 'height': 96.5, 'sign': 0}, {'coordinates': [292.0, 328.0, 100.5], 'original_index': 20514, 'new_index': 322, 'height': 100.5, 'sign': 0}, {'coordinates': [289.0, 334.0, 102.5], 'original_index': 19459, 'new_index': 433, 'height': 102.5, 'sign': 0}, {'coordinates': [303.0, 277.0, 103.5], 'original_index': 23535, 'new_index': 540, 'height': 103.5, 'sign': 0}, {'coordinates': [311.0, 274.0, 103.5], 'original_index': 25143, 'new_index': 541, 'height': 103.5, 'sign': 0}, {'coordinates': [281.0, 318.0, 106.5], 'original_index': 17536, 'new_index': 830, 'height': 106.5, 'sign': 0}, {'coordinates

In [13]:
intervals = compute_intervals(births, mergers)

In [16]:
print(intervals)

[[93.5, -1], [94.5, 98.5], [95.5, 112.5], [96.5, 103.5], [100.5, 108.5], [102.5, 103.5], [103.5, 129.5], [103.5, 107.5], [106.5, 107.5], [107.5, 113.5], [107.5, 127.5], [113.5, 123.5], [114.5, 170.5], [116.5, 119.5], [120.5, -1], [120.5, 149.5], [121.5, 123.5], [123.5, -1], [125.5, 126.5], [126.5, 144.5], [129.5, 132.5], [130.5, 131.5], [132.5, 145.5], [132.5, 133.5], [133.5, 134.5], [136.5, 137.5], [137.5, 139.5], [140.5, 146.5], [140.5, 145.5], [140.5, 141.5], [141.5, 143.5], [142.5, 143.5], [150.5, 154.5], [152.5, 153.5], [152.5, -1], [156.5, -1], [157.5, -1], [172.5, -1]]


In [14]:
def compute_largest_bar(intervals):
    largest_bar = 0
    for interval in intervals:
        if length_of_interval(interval) > largest_bar:
            largest_bar = length_of_interval(interval)
    return largest_bar

In [15]:
print(compute_largest_bar(intervals))

56.0
